# Игра "камень, ножницы, бумага"

Установка необходимых библиотек

In [ ]:
%pip install -q -U pandas

In [ ]:
%pip install -q -U matplotlib

In [ ]:
%pip install -q -U seaborn

In [ ]:
%pip install -q -U kaggle_environments

Импорт библиотек и функций

In [1]:
import numpy as np
import pandas as pd
import random

import matplotlib.pyplot as plt
import seaborn as sns

from kaggle_environments import make, evaluate


Ходы игроков представлены целочисленными константами:
* 0 - "камень"
* 1 - "бумага"
* 2 - "ножницы"

Функция, возвращающая ход, который нужно сделать, чтобы выиграть оппонента, на основании его предполагаемого хода _opponent_action_

In [2]:
def action_to_win(opponent_action):
    if opponent_action == 0:
        return 1
    elif opponent_action == 1:
        return 2
    elif opponent_action == 2:
        return 0
    else:
        return -1


Атрибут _observation.lastOpponentAction_ присутствует не всегда, поэтому в функциях агентов делается проверка его существования вызовом _hasattr(obj, name)_

Примеры игровых агентов

In [17]:
# агент, выдающий всегда "камень"
def rock_agent(observation, configuration):
    return 0


# агент, выдающий всегда "бумага"
def paper_agent(observation, configuration):
    return 1


# агент, выдающий всегда "ножницы"
def scissors_agent(observation, configuration):
    return 2


# агент, копирующий предыдущий ход оппонента
def copy_agent(observation, configuration):
    if observation.step > 0:
        return observation.lastOpponentAction
    else:
        return random.randrange(0, configuration.signs)


# агент, предполагающий, что оппонент меняет свой ход на каждом шаге
def reactionary_agent(observation, configuration):
    if observation.step > 0:
        expected_opponent_actions = [0, 1, 2]
        expected_opponent_actions.remove(observation.lastOpponentAction)
        expected_opponent_action = expected_opponent_actions[random.randrange(0, 2)]
        return action_to_win(expected_opponent_action)
    else:
        return random.randrange(0, configuration.signs)


# агент, играющий поочерёдно "камень", "бумага", "ножницы"
seq_opponent_agent_action = None
def sequental_agent(observation, configuration):
    global seq_opponent_agent_action
    if observation.step == 0:
        seq_opponent_agent_action = random.randrange(0, configuration.signs)
    seq_opponent_agent_action = (seq_opponent_agent_action + 1) % configuration.signs
    return seq_opponent_agent_action


# агент, выдающий на каждом шаге случайный ход
def random_agent(observation, configuration):
    return random.randrange(0, configuration.signs)


# агент, выбирающий случайно или "камень", или "бумага"
def rock_paper_agent(observation, configuration):
    possible_action = random.randrange(0, 2)
    return 0 if possible_action == 0 else 1


# агент, выбирающий случайно или "камень", или "ножницы"
def rock_scissors_agent(observation, configuration):
    possible_action = random.randrange(0, 2)
    return 0 if possible_action == 0 else 2


# агент, выбирающий случайно или "бумага", или "ножницы"
def paper_scissors_agent(observation, configuration):
    possible_action = random.randrange(0, 2)
    return 1 if possible_action == 0 else 2


# агент, копящий статистику всех ходов оппонента, определяющий его самый частый ход
# и выбирающий ход, чтобы победить с наибольшей вероятностью
stat_opp_action_counts = None
def statistical_agent(observation, configuration):
    global stat_opp_action_counts
    if observation.step == 0:
        stat_opp_action_counts = [0, 0, 0]
    if hasattr(observation, 'lastOpponentAction') and \
    observation.lastOpponentAction in list(range(0, configuration.signs)):
        stat_opp_action_counts[observation.lastOpponentAction] += 1
    frequent_opponent_action = stat_opp_action_counts.index(max(stat_opp_action_counts))
    return action_to_win(frequent_opponent_action)


# агент, копящий статистику последних 10 ходов оппонента, определяющий его самый частый ход
# за эти 10 ходов и выбирающий ход, чтобы победить с наибольшей вероятностью
react_stat_opp_actions = None
def react_statistical_agent(observation, configuration):
    global react_stat_opp_actions
    if observation.step == 0:
        react_stat_opp_actions = []
    if hasattr(observation, 'lastOpponentAction'):
        react_stat_opp_actions.append(observation.lastOpponentAction)
    if (len(react_stat_opp_actions) > 10):
        react_stat_opp_actions = react_stat_opp_actions[1:]
    opponent_action_counts = [react_stat_opp_actions.count(action) for action in range(0, configuration.signs)]
    frequent_opponent_action = opponent_action_counts.index(max(opponent_action_counts))
    return action_to_win(frequent_opponent_action)


Пример игры между агентами

In [89]:
evaluate(
    environment="rps", # выбор игровой среды "камень, ножницы, бумага"
    configuration={"episodeSteps", 100}, #количество ходов игровых агентов
    agents=[rock_agent, scissors_agent], #игровые агенты
    num_episodes=1 #число эпизодов
)


[[999.0, -999.0]]

Прогон игр между различными агентами

In [45]:
evaluate(
    environment="rps", # выбор игровой среды "камень, ножницы, бумага"
    configuration={"episodeSteps", 100}, #количество ходов игровых агентов
    agents=[statistical_agent, sequental_agent], #игровые агенты
    num_episodes=1, #число эпизодов
)

[[0, 0]]

In [53]:
evaluate(
    environment="rps", # выбор игровой среды "камень, ножницы, бумага"
    configuration={"episodeSteps", 100}, #количество ходов игровых агентов
    agents=[statistical_agent, random_agent], #игровые агенты
    num_episodes=1, #число эпизодов
)

[[0, 0]]

In [38]:
evaluate(
    environment="rps", # выбор игровой среды "камень, ножницы, бумага"
    configuration={"episodeSteps", 100}, #количество ходов игровых агентов
    agents=[react_statistical_agent, statistical_agent], #игровые агенты
    num_episodes=1, #число эпизодов
)

[[151.0, -151.0]]

По итогам прогонов игр между разными агентами наилучшие результаты показывают агенты _statistical_agent_ и _react_statistical_agent_